In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [4]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_20200420_contact_tracing_covid_with_annex_en.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/covid_location.md"

In [6]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 2, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [1]:
import re
import pandas as pd
file_path = "../original/covid_location.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [2]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX -- (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4])
df.reset_index(inplace=True)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [3]:
df[60:80]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,index,section,subsection,point,heading,text,section_reference
60,65,,,,False,"47. The controller, in collaboration with the ...",3.2
61,66,4,,,True,CONCLUSION,4
62,67,,,,False,48. The world is facing a significant public h...,4
63,68,,,,False,49. The EDPB underlines that one should not ha...,4
64,69,Annex,,,True,CONTACT TRACING APPLICATIONS ANALYSIS GUIDE,Annex
65,70,,,,False,0. Disclaimer,Annex
66,71,,,,False,The following guidance is neither prescriptive...,Annex
67,72,,,,False,It must also be noted that this guide is of a ...,Annex
68,73,,,,False,1. Summary,Annex
69,74,,,,False,In many Member States stakeholders are conside...,Annex


In [4]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 9 augmented with footnotes
Row 10 augmented with footnotes
Row 16 augmented with footnotes
Row 17 augmented with footnotes
Row 18 augmented with footnotes
Row 19 augmented with footnotes
Row 25 augmented with footnotes
Row 27 augmented with footnotes
Row 34 augmented with footnotes
Row 42 augmented with footnotes
Row 44 augmented with footnotes
Row 45 augmented with footnotes
Row 51 augmented with footnotes
Row 53 augmented with footnotes
Row 55 augmented with footnotes


In [5]:
file = "../inputs/documents/covid_location.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [14]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.covid_location
importlib.reload(gdpr_rag.documents.covid_location)
from gdpr_rag.documents.covid_location import CovidLocation

path_to_manual_as_csv_file = "../inputs/documents/covid_location.parquet"
doc = CovidLocation(path_to_manual_as_csv_file)


In [15]:
from IPython.display import Markdown, display

section = "1"

section = "2.1"
#section = "Annex"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


2 USE OF LOCATION DATA
2.1 Sources of location data


# 2 USE OF LOCATION DATA

## 2.1 Sources of location data

9. There are two principal sources of location data available for modelling the spread of the virus and the overall effectiveness of  confinement measures:

- location data collected by electronic communication service providers (such as mobile telecommunication operators) in the course of the provision of their service ; and

- location data collected by information society service providers' applications whose functionality requires the use of such data (e.g., navigation, transportation services, etc.).

10. The EDPB recalls that location data[^4] collected from electronic communication providers may only be processed within the remits of articles 6 and 9 of the ePrivacy Directive. This means that these data can only be transmitted to authorities or other third parties if they have been anonymised by the provider or, for data indicating the geographic position of the terminal equipment of a user, which are not traffic data, with the prior consent of the users[^5].

11. Regarding information, including location data, collected directly from the terminal equipment, art. 5(3) of the "ePrivacy" directive applies. Hence, the storing of information on the user's device or gaining access to the information already stored is allowed only if (i) the user has given consent[^6] or (ii) the storage and/or access is strictly necessary for the information society service explicitly requested by the user.

12. Derogations to the rights and obligations provided for in the "ePrivacy" Directive are however possible pursuant to Art. 15, when they constitute a necessary, appropriate and proportionate measure within a democratic society for certain objectives[^7].

13. As for the re-use of location data collected by an information society service provider for modelling purposes (e.g., through the operating system or some previously installed application) additional conditions must be met. Indeed, when data have been collected in compliance with Art. 5(3) of the ePrivacy Directive, they can only be further processed with the additional consent of the data subject or on the basis of a Union or Member State law which constitutes a necessary and proportionate measure in a democratic society to safeguard the objectives referred to in Art. 23 (1) GDPR.[^8]

  
[^4]: See Art. 2(c) of the ePrivacy Directive.  
[^5]: See Art 6 and 9 of the ePrivacy Directive.  
[^6]: The notion of consent in the ePrivacy directive remains the notion of consent in the GDPR and must meet all the requirements of the consent as provided by art. 4(11) and 7 GDPR  
[^7]: For the interpretation of article 15 of the "ePrivacy" Directive, see also, CJEU Judgment of 29 January 2008 in case C-275/06, Productores de Música de España (Promusicae) v. Telefónica de España SAU.  
[^8]: See section 1.5.3 of the guidelines 1/2020 on processing personal data in the context of connected vehicles.